In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import ast

import gurobipy as gp
from gurobipy import GRB, Model
import math

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("raw.csv", parse_dates=['date'])
df_information = pd.read_csv("info.csv")

In [3]:
start_date = pd.Timestamp('2022-12-31')
end_date = pd.Timestamp('2023-12-31')

filtered_df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]

In [5]:
filtered_df = filtered_df.sort_values(by='date').reset_index(drop=True)
filtered_df

,date,ticker,adj_close
0,2023-01-02,PJAA.JK,683.08
1,2023-01-02,BEKS.JK,50.00
2,2023-01-02,BKSL.JK,50.00
3,2023-01-02,BLTA.JK,50.00
4,2023-01-02,BTEK.JK,50.00
...,...,...,...
210086,2023-12-29,MDIA.JK,50.00
210087,2023-12-29,MARI.JK,50.00
210088,2023-12-29,MARI.JK,50.00
210089,2023-12-29,POOL.JK,50.00


# Monthly Returns

In [6]:
tickers = filtered_df['ticker'].unique()

In [7]:
# new df
monthly_returns = pd.DataFrame(columns=['ticker', 'month', 'monthly_return_percentage'])

for ticker in tickers:
    ticker_data = filtered_df[filtered_df['ticker'] == ticker]
    for month in range(1, 13):
        month_data = ticker_data[(ticker_data['date'].dt.year == 2023) & (ticker_data['date'].dt.month == month)]
        if not month_data.empty:
            adj_close_start = month_data.iloc[0]['adj_close']  # awal bulan
            adj_close_end = month_data.iloc[-1]['adj_close']   # akhir bulan

            # persentase bulanan return
            monthly_return_percentage = ((adj_close_end - adj_close_start) / adj_close_start) * 100
            
            temp_df = pd.DataFrame({
                'ticker': [ticker],
                'month': [f'2023-{month:02}'],
                'monthly_return_percentage': [monthly_return_percentage]
            })

            monthly_returns = pd.concat([monthly_returns, temp_df], ignore_index=True)

In [8]:
monthly_returns

,ticker,month,monthly_return_percentage
0,PJAA.JK,2023-01,-2.816654
1,PJAA.JK,2023-02,-1.449144
2,PJAA.JK,2023-03,0.000000
3,PJAA.JK,2023-04,1.428444
4,PJAA.JK,2023-05,2.554153
...,...,...,...
10439,BABY.JK,2023-12,-15.714286
10440,AEGS.JK,2023-09,-32.631579
10441,AEGS.JK,2023-10,-15.873016
10442,AEGS.JK,2023-11,18.367347


In [9]:
# monthly_returns.to_csv("tickers_monthly_returns_percentage_2023.csv", index=False)